## Secure Applications

In this notebook you will: 

-   Reserve resources for this experiment
-   Configure your reserved resources
-   Access your reserved resources over SSH
-   Retrieve files saved on a FABRIC resources
-   Delete your FABRIC reservation (in case you finish early)

### Exercise: Reserve resources

In this exercise, we will reserve resources on FABRIC for the experiments on network layer security.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

In [ ]:
import os
slice_name="network-security_" + os.getenv('NB_USER')

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

Next, we’ll select a random FABRIC site for our experiment. We’ll make sure to get one that has sufficient capacity for the experiment we’re going to run.

Once we find a suitable site, we’ll print details about available resources at this site.

In [ ]:
exp_requires = {'core': 3*2, 'nic': 4}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break

fablib.show_site(site_name)

In [ ]:
# this cell sets up the nodes
node_names = ["romeo", "juliet", "server", "router-int", "router-ext", "vpn"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=2, ram=4, disk=10, image='default_ubuntu_22')

In [ ]:
# this cell sets up the network links
nets = [
    {"name": "net0",   "nodes": ["romeo", "router-int"]},
    {"name": "net1",  "nodes": ["server", "router-int"]},
    {"name": "net2",  "nodes": ["router-int", "vpn"]},
    {"name": "net3",  "nodes": ["vpn", "router-ext"]},
    {"name": "net4",  "nodes": ["router-ext", "juliet"]}
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

While it is being prepared, the “State” of the slice will appear as “Configuring”.
When it is ready, the “State” of the slice will change to “StableOK”.

In [ ]:
slice.submit()

In [ ]:
slice.wait_ssh(progress=True)

### Exercise: Configure resources

Next, we need to configure our resources - assign IP addresses to network interfaces, enable forwarding on the router, and install any necessary software.

First, we’ll configure IP addresses and add the IP addresses and hostnames to the host files:

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf = {
    "romeo-net0-p1":       {"addr": "10.10.1.100", "subnet": "10.10.1.0/24", "hostname": "romeo"},
    "router-int-net0-p1":  {"addr": "10.10.1.1", "subnet": "10.10.1.0/24", "hostname": "router-int"},
    "router-int-net1-p1":  {"addr": "10.10.2.1", "subnet": "10.10.2.0/24", "hostname": "router-int"},
    "server-net1-p1":      {"addr": "10.10.2.100", "subnet": "10.10.2.0/24", "hostname": "server"},
    "router-int-net2-p1":  {"addr": "10.10.3.1", "subnet": "10.10.3.0/24", "hostname": "router-int"},
    "vpn-net2-p1":         {"addr": "10.10.3.100", "subnet": "10.10.3.0/24", "hostname": "vpn"},
    "vpn-net3-p1":         {"addr": "10.10.4.100", "subnet": "10.10.4.0/24", "hostname": "vpn"},
    "router-ext-net3-p1":  {"addr": "10.10.4.1", "subnet": "10.10.4.0/24", "hostname": "router-ext"},
    "router-ext-net4-p1":  {"addr": "10.10.5.1", "subnet": "10.10.5.0/24", "hostname": "router-ext"},
    "juliet-net4-p1":      {"addr": "10.10.5.100", "subnet": "10.10.5.0/24", "hostname": "juliet"}
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))


command = ""
for if_name in if_conf:
    command = command + f"echo '{if_conf[if_name]['addr']}\t{if_conf[if_name]['hostname']}' | sudo tee -a /etc/hosts > /dev/null; "
for node in node_names:
    slice.get_node(name=node).execute(command)

Let’s make sure that all of the network interfaces are brought up:

In [ ]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

Then, we’ll add routes on hosts so that hosts know how to reach each other.

In [ ]:
rt_conf = [
    {"name": "romeo", "addr": "10.0.0.0/8", "gw": "10.10.1.1"},
    {"name": "server", "addr": "10.0.0.0/8", "gw": "10.10.2.1"},
    {"name": "router-int", "addr": "10.0.0.0/8", "gw": "10.10.3.100"},
    {"name": "vpn", "addr": "10.0.0.0/8", "gw": "10.10.3.1"},
    {"name": "vpn", "addr": "10.10.4.0/22", "gw": "10.10.4.1"},
    {"name": "router-ext", "addr": "10.0.0.0/8", "gw": "10.10.4.100"},
    {"name": "juliet", "addr": "10.0.0.0/8", "gw": "10.10.5.1"}
]
for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])


Finally, we’ll install some software on nodes.

In [ ]:
for n in ['vpn', 'juliet']:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install openvpn", quiet=True)
for n in ['juliet', 'server', 'router-int', 'router-ext']:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install net-tools", quiet=True)

### Exercise: Network Topology
Now, we can use the following code to draw the topology of this network.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
edges = [(iface.get_network().get_name(), iface.get_node().get_name()) for iface in slice.get_interfaces()]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
plt.figure(figsize=(12,12)) # or some other size 
nx.draw(G, node_size=2000, node_shape='s', node_color=[n[1]['color'] for n in nodes], with_labels=True)

### Exercise: Log in to resources
Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the nodes in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the nodes as follows:

-   In Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   Copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each host and the router. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

Now you can continue to perform the Network Layer Security experiments on these host sessions.

### Exercise: Transfer .pcap files from a FABRIC host


Now, retrieve the `vpn-router-int.pcap` and `vpn-router-ext.pcap` files from router-int and router-ext using the following commands:

In [ ]:
host_name = "router-int"
slice.get_node(host_name).download_file("/home/fabric/work/vpn-%s.pcap" % host_name, "/home/ubuntu/vpn-%s.pcap" % host_name)

In [ ]:
host_name = "router-ext"
slice.get_node(host_name).download_file("/home/fabric/work/vpn-%s.pcap" % host_name, "/home/ubuntu/vpn-%s.pcap" % host_name)

Then in the Jupyter environment, click on the folder icon in the file browser on the left to make sure that you are located in your “Jupyter home” directory.

Then, you should see the above .pcap files appear in the Jupyter file browser on the left. You can now download this file from the Jupyter environment to your own laptop to analyze them in Wireshark.

### Delete your slice resources

If you finished your experiment early, you should delete your slice! The following cell deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice.delete()

In [ ]:
slice.show()